In [46]:
import os

In [47]:
from menpodetect import load_opencv_frontal_face_detector
opencv_detector = load_opencv_frontal_face_detector()

In [48]:
def load_data(path_to_images, path_to_labels):
    images = []
    labels = []
    (image_paths, raw_labels) = get_files(path_to_images, path_to_labels)

    for i in range(len(image_paths)):
        try:
            image = load_image(image_paths[i])
            _ = load_vector(image)
            images.append(image)
            labels.append(raw_labels[i])
        except ValueError:
#             print("Not squared")
            continue

    return (images, labels)

In [49]:
def get_files(images_path, labels_path):
    images_dirs = get_dirs(images_path)
    images = []
    labels = []

    for dir in images_dirs:
        try:
            dir_path = os.path.join(dir.split(os.sep)[-2], dir.split(os.sep)[-1])
            label = os.listdir(os.path.join(labels_path, dir_path))[0]
            label_dir = os.path.join(os.path.join(labels_path, dir_path), label)
            label_f = open(label_dir, "r")
            line = label_f.readline()
            labels.append(int(float(line.rstrip())))

            image = sorted(os.listdir(dir))[-1]
            image_path = os.path.join(dir, image)
            images.append(image_path)
        except FileNotFoundError:
            continue
        except IndexError:
            continue

    return (images, labels)

In [50]:
def get_dirs(path):
    sub_dirs = [os.path.join(path, o) for o in os.listdir(path) 
                    if os.path.isdir(os.path.join(path,o))]
    if (len(sub_dirs) == 0):
        return [path]
    else:
        l = []
        for sub_dir in sub_dirs:
            l.extend(get_dirs(sub_dir))
        return l

In [51]:
from scipy import misc

def load_image(path_to_image):
    image = misc.imread(path_to_image, mode='L')
    image = misc.imresize(image, [400, 400])
    return image

In [52]:
import menpo
from menpo.feature import vector_128_dsift

def load_vector(image):
    image = menpo.image.Image(image)
    if image.n_channels != 1:
        image = image.as_greyscale()
    opencv_detector(image)
    try:
        image = image.crop_to_landmarks_proportion(0.2).rescale_landmarks_to_diagonal_range(400)
    except:
#         print("No face detected")
        None
    vector = vector_128_dsift(image).reshape(128)
    return vector

# Main

In [53]:
# Load data
(images, labels) = load_data(path_to_images=os.path.normpath("data\ck+\extended-cohn-kanade-images\cohn-kanade-images"),
                             path_to_labels=os.path.normpath("data\ck+\Emotion_labels\Emotion"))

C:\Users\hoipc\AppData\Local\conda\conda\envs\conda_env\lib\site-packages\menpo\image\base.py:2685: MenpoDeprecationWarning: This method is no longer supported and will be removed in a future version of Menpo. Use .pixels_with_channels_at_back() instead.
  MenpoDeprecationWarning)


In [54]:
from sklearn.model_selection import train_test_split

# Split data
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2)

In [55]:
from sklearn import svm
from sklearn.pipeline import Pipeline
from skimage.color import rgb2gray

# Define PipeStep class
class PipeStep(object):
        """
        Wrapper for turning functions into pipeline transforms (no-fitting)
        """
        def __init__(self, step_func):
            self._step_func=step_func
        def fit(self,*args):
            return self
        def transform(self,X):
            return self._step_func(X)

# Define steps
makegray_step = PipeStep(lambda img_list: [rgb2gray(img) for img in img_list])
vectorize_step = PipeStep(lambda img_list: [load_vector(img) for img in img_list])

# Construct Pipeline
classifier = Pipeline([('Makegray', makegray_step),
                       ('Vectorize', vectorize_step),
                       ('SVM', svm.SVC(kernel='linear', probability=True))])

In [56]:
# Train
classifier.fit(X_train, y_train)

C:\Users\hoipc\AppData\Local\conda\conda\envs\conda_env\lib\site-packages\menpo\image\base.py:2685: MenpoDeprecationWarning: This method is no longer supported and will be removed in a future version of Menpo. Use .pixels_with_channels_at_back() instead.
  MenpoDeprecationWarning)


Pipeline(memory=None,
     steps=[('Makegray', <__main__.PipeStep object at 0x000002174B4806D8>), ('Vectorize', <__main__.PipeStep object at 0x000002174B480780>), ('SVM', SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=True, random_state=None, shrinking=True,
  tol=0.001, verbose=False))])

In [57]:
from sklearn.metrics import classification_report

# Test
pred_test = classifier.predict(X_test)
pred_prop = classifier.predict_proba(X_test)
print(classification_report(y_true=y_test, y_pred=pred_test))

C:\Users\hoipc\AppData\Local\conda\conda\envs\conda_env\lib\site-packages\menpo\image\base.py:2685: MenpoDeprecationWarning: This method is no longer supported and will be removed in a future version of Menpo. Use .pixels_with_channels_at_back() instead.
  MenpoDeprecationWarning)


             precision    recall  f1-score   support

          1       0.17      0.29      0.21         7
          2       0.67      0.25      0.36         8
          3       0.70      0.78      0.74         9
          4       0.33      0.25      0.29         4
          5       0.88      0.82      0.85        17
          6       0.00      0.00      0.00         2
          7       0.85      0.69      0.76        16

avg / total       0.68      0.59      0.61        63



In [58]:
# Predict
image = load_image('data/test3-7.png')
pred_prop_1 = classifier.predict_proba([image])
print(pred_prop_1)

[[ 0.02656171  0.03228368  0.01333562  0.05082742  0.00367976  0.21051479
   0.66279702]]


C:\Users\hoipc\AppData\Local\conda\conda\envs\conda_env\lib\site-packages\menpo\image\base.py:2685: MenpoDeprecationWarning: This method is no longer supported and will be removed in a future version of Menpo. Use .pixels_with_channels_at_back() instead.
  MenpoDeprecationWarning)


In [59]:
import lime
from lime import lime_image
from lime.wrappers.scikit_image import SegmentationAlgorithm

# Explain
explainer = lime_image.LimeImageExplainer(verbose = False)
segmenter = SegmentationAlgorithm('slic', n_segments=100, compactness=1, sigma=1)
explanation = explainer.explain_instance(image, 
                                         classifier_fn = classifier.predict_proba, 
                                         top_labels=3, hide_color=0, num_samples=1000, segmentation_fn=segmenter)

C:\Users\hoipc\AppData\Local\conda\conda\envs\conda_env\lib\site-packages\menpo\image\base.py:2652: MenpoDeprecationWarning: This method is no longer supported and will be removed in a future version of Menpo. Use .pixels_with_channels_at_back instead.
  MenpoDeprecationWarning)


In [60]:
# now show them for each class
fig, m_axs = plt.subplots(2,3, figsize = (12,4))
for i, (c_ax, gt_ax) in zip(explanation.top_labels, m_axs.T):
    temp, mask = explanation.get_image_and_mask(i, positive_only=True, num_features=5, hide_rest=False, min_weight=0.01)
    c_ax.imshow(label2rgb(mask,temp, bg_label = 0), interpolation = 'nearest')
    c_ax.set_title('Positive for {}\nScore:{:2.2f}%'.format(i+1, 100*pred_prop_1[0, i]))
    c_ax.axis('off')
    face_id = np.random.choice(np.where(y_train==i+1)[0])
    gt_ax.imshow(X_train[face_id])
    gt_ax.set_title('Example of {}'.format(i+1))
    gt_ax.axis('off')

AttributeError: module 'matplotlib.colors' has no attribute 'to_rgba'